In [1]:
from src.extract import extract
from src.export import export, export_dir, display_progress

from slippi import Game

from pymongo import MongoClient
from bson.binary import Binary
from scipy import sparse
import numpy as np
import pickle

In [2]:
# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client['delme']
collection = db['test']

In [3]:
# export all .slp files in dir
dir_full_path = '/media/zack/vault/slippis/TONG#898/'
export_dir(dir_full_path, 'delme', 'test')

[####################] 11 of 11 - 100.0%

In [4]:
# number of documents in database
# each game produces two documens,
# one for each player
collection.count_documents({})

22

In [5]:
# retreive player data on single game from database
p1, p2 = tuple(collection.find({'game_id':'Game_20201119T165940.slp'}))[:2]

In [6]:
# convert istream from "sparse matrix bytestream" to numpy array
print(type(p1['istream']))
if isinstance(p1['istream'], bytes):
    p1['istream'] = pickle.loads( p1['istream']).toarray()
    p2['istream'] = pickle.loads( p2['istream']).toarray()
print(type(p1['istream']))

<class 'bytes'>
<class 'numpy.ndarray'>


In [7]:
print(f"{p1['name']}: {p1['character']}")
print(f"{p2['name']}: {p2['character']}")

X0v3r: MARTH
#Free Melee: CAPTAIN_FALCON


In [8]:
def presses(buttonstream):
    return np.sum(np.abs(np.diff(buttonstream, axis=0)))/2

In [9]:
resultA = presses(p1['istream'][:, 9]) > presses(p2['istream'][:, 9])
resultB = presses(p1['istream'][:, 8]) > presses(p2['istream'][:, 8])
resultXY = presses(p1['istream'][:, [6,7]]) > presses(p2['istream'][:, [6,7]])
resultLR = presses(p1['istream'][:, [10,11]]) > presses(p2['istream'][:, [10,11]])
resultZ = presses(p1['istream'][:, 12]) > presses(p2['istream'][:, 12])

whoA = p1 if resultA else p2
whoB = p1 if resultB else p2
whoXY = p1 if resultXY else p2
whoLR = p1 if resultLR else p2
whoZ = p1 if resultZ else p2

print(f"Match Length: {round(len(p1['istream'])/3600, 2)} minutes")
print(f"Match ID: {p1['game_id']}")

print(f"\nMost A presses: {presses(whoA['istream'][:, 9])}\n --- {whoA['name']} - {whoA['character']}")
print(f"\nMost B presses: {presses(whoB['istream'][:, 8])}\n --- {whoB['name']} - {whoB['character']}")
print(f"\nMost X or Y presses: {presses(whoXY['istream'][:, [6,7]])}\n --- {whoXY['name']} - {whoXY['character']}")
print(f"\nMost L or R presses: {presses(whoLR['istream'][:, [10,11]])}\n --- {whoLR['name']} - {whoLR['character']}")
print(f"\nMost Z presses: {presses(whoZ['istream'][:, 12])}\n --- {whoZ['name']} - {whoZ['character']}")

Match Length: 3.22 minutes
Match ID: Game_20201119T165940.slp

Most A presses: 127.0
 --- X0v3r - MARTH

Most B presses: 31.0
 --- X0v3r - MARTH

Most X or Y presses: 183.0
 --- X0v3r - MARTH

Most L or R presses: 110.0
 --- #Free Melee - CAPTAIN_FALCON

Most Z presses: 15.0
 --- #Free Melee - CAPTAIN_FALCON


In [10]:
if presses(p1['istream'][:, 6]) > presses(p1['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p1['name']} uses {jump_button} to jump")

if presses(p2['istream'][:, 6]) > presses(p2['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p2['name']} uses {jump_button} to jump")

X0v3r uses X to jump
#Free Melee uses Y to jump
